In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import RELACS as rel
sns.set_style("white")
sns.set_context("paper")
%load_ext rpy2.ipython
%matplotlib inline

In [2]:
%%R

                                    #########################################
                                    #### GO TERM OVER-REPRESENTATION TEST ###
                                    #########################################

### LOAD PACKAGES ###
library(clusterProfiler)
library(org.Mm.eg.db)
library(dplyr)
library(dplyr)



### LOAD DIFFERENTIAL EXPRESSION ANALYSIS RESULTS ###
esc_de = read.csv("../INTERMEDIATE_FILES/DE_genes_shrinked_apeglm_mESC_EPZvsDMSO.tsv", sep="\t")
row.names(esc_de) = gsub("\\.[0-9]+","",row.names(esc_de))

npc_de = read.csv("../INTERMEDIATE_FILES/DE_genes_shrinked_apeglm_NPC48h_EPZvsDMSO.tsv", sep="\t")
row.names(npc_de) = gsub("\\.[0-9]+","",row.names(npc_de))


### SELECT UP-REGULATED AND DOWN-REGULATED GENES FOR EACH 
esc_up = row.names(na.omit(esc_de[(esc_de$padj < 0.05) & (esc_de$log2FoldChange > 0), ]))
esc_down = row.names(na.omit(esc_de[(esc_de$padj < 0.05) & (esc_de$log2FoldChange < 0), ]))

npc_up = row.names(na.omit(npc_de[(npc_de$padj < 0.1) & (npc_de$log2FoldChange > 0), ]))
npc_down = row.names(na.omit(npc_de[(npc_de$padj < 0.1) & (npc_de$log2FoldChange < 0), ]))


### TRANSLATE GENE ID TO ENTREZ ID ###
esc_up_tr = bitr(esc_up, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
esc_down_tr = bitr(esc_down, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
uni_esc = bitr(row.names(na.omit(esc_de)), fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")

npc_up_tr = bitr(npc_up, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
npc_down_tr = bitr(npc_down, fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
uni_npc = bitr(row.names(na.omit(npc_de)), fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")


/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: clusterProfiler v3.10.1  For help: https://guangchuangyu.github.io/software/clusterProfiler

If you use clusterProfiler in published research, please cite:
Guangchuang Yu, Li-Gen Wang, Yanyan Han, Qing-Yu He. clusterProfiler: an R package for comparing biological themes among gene clusters. OMICS: A Journal of Integrative Biology. 2012, 16(5):284-287.

  warnings.warn(x, RRuntimeWarning)
/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: AnnotationDbi

  warnings.warn(x, RRuntimeWarning)
/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: stats4

  warnings.warn(x, RRuntimeWarning)
/home/ferrari/.local/lib/pyt

In [16]:
%%R

### PERFORM OVER-REPRESENTATION TEST FOR mESC ###
list_def_esc = list(deg_up = esc_up_tr$ENTREZID, deg_down = esc_down_tr$ENTREZID)

ck_esc <- compareCluster(geneCluster = list_def_esc,
                     universe = uni_esc$ENTREZID,
                     fun = "enrichGO",
                     OrgDb = "org.Mm.eg.db",
                     ont = "BP",
                     pAdjustMethod = "BH",
                     qvalueCutoff = 0.05,
                     pvalueCutoff = 0.05,
                     readable = TRUE)

df_ck_esc = as.data.frame(ck_esc)

In [38]:
%%R

### PERFORM OVER-REPRESENTATION TEST FOR NPC48h ###
list_def_npc = list(deg_up = npc_up_tr$ENTREZID, deg_down = npc_down_tr$ENTREZID)

ck_npc <- compareCluster(geneCluster = list_def_npc,
                     universe = uni_npc$ENTREZID,
                     fun = "enrichGO",
                     OrgDb = "org.Mm.eg.db",
                     ont = "BP",
                     pAdjustMethod = "BH",
                     qvalueCutoff = 0.05,
                     pvalueCutoff = 0.05,
                     readable = TRUE)

df_ck_npc = as.data.frame(ck_npc)

In [25]:
%%R
pdf("../FIGURES/GO_mESC_EPZvsDMSO_compareClusters.pdf", width=7.5,height=5)
p = dotplot(ck_esc, showCategory=10)
print(p)
dev.off()

png 
  2 


In [39]:
%%R
pdf("../FIGURES/GO_NPC48h_EPZvsDMSO_compareClusters.pdf", width=7.5,height=5)
p = dotplot(ck_npc, showCategory=10)
print(p)
dev.off()

png 
  2 


In [22]:
%%R

### GSEA WIKIPATHWAY mESC ###

## PREPARE DATA ##

esc_de$GeneID = rownames(esc_de)
universe = bitr(row.names(esc_de), fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
universe  = merge(as.data.frame(universe),as.data.frame(esc_de), by.x = "ENSEMBL", by.y = "GeneID")
universe = universe[!is.na(universe$log2FoldChange),]
universe = universe[!duplicated(universe$ENTREZID), ]
universe = universe[order(universe$log2FoldChange, decreasing = TRUE), ]

geneList_ESC = universe$log2FoldChange
names(geneList_ESC) = universe$ENTREZID


In [25]:
%%R

### GSEA WIKIPATHWAY mESC ###

wp2gene = read.csv("/data/manke/group/shiny/ferrari/Genes2Functions/shared_files/wikipath_mouse.gmt",sep="\t")
wp2gene = na.omit(wp2gene)
      
wp2gene <- wp2gene %>% tidyr::separate(ont, c("name","version","wpid","org"), "%")
wpid2gene <- wp2gene %>% dplyr::select(wpid, gene) #TERM2GENE
wpid2name <- wp2gene %>% dplyr::select(wpid, name) #TERM2NAME

ewp_esc <- GSEA(geneList_ESC, 
                   pvalueCutoff = 0.05,
                   pAdjustMethod ="BH",
                   TERM2GENE = wpid2gene, 
                   TERM2NAME = wpid2name, 
                   minGSSize    = 20,
                   maxGSSize    = 700,
                   nPerm = 10000,
                   verbose=FALSE)

ewp_esc_read <- setReadable(ewp_esc, "org.Mm.eg.db", keyType = "ENTREZID")
ewp_esc_df = as.data.frame(ewp_esc_read)
write.table(ewp_esc_df,"../INTERMEDIATE_FILES/pathway_mESC_EPZvsDMSO.tsv", sep="\t",quote=F)


In [26]:
%%R

pdf("../FIGURES/GSEA_ESC_EPZvsDMSO_WIKIPATHWAY.pdf", width=10,height=5)
p = ridgeplot(ewp_esc, showCategory=15)
print(p)
dev.off()

/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Picking joint bandwidth of 0.0552

  warnings.warn(x, RRuntimeWarning)


png 
  2 


In [4]:
%%R

### GSEA WIKIPATHWAY NPC48h ###

## PREPARE DATA ##

npc_de$GeneID = rownames(npc_de)
universe = bitr(row.names(npc_de), fromType="ENSEMBL", toType=c("ENTREZID","ENSEMBL","SYMBOL"), OrgDb="org.Mm.eg.db")
universe  = merge(as.data.frame(universe),as.data.frame(npc_de), by.x = "ENSEMBL", by.y = "GeneID")
universe = universe[!is.na(universe$log2FoldChange),]
universe = universe[!duplicated(universe$ENTREZID), ]
universe = universe[order(universe$log2FoldChange, decreasing = TRUE), ]

geneList_NPC = universe$log2FoldChange
names(geneList_NPC) = universe$ENTREZID


/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 'select()' returned 1:many mapping between keys and columns

  warnings.warn(x, RRuntimeWarning)


In [6]:
%%R

### GSEA WIKIPATHWAY NPC48h ###

wp2gene = read.csv("/data/manke/group/shiny/ferrari/Genes2Functions/shared_files/wikipath_mouse.gmt",sep="\t")
wp2gene = na.omit(wp2gene)
      
wp2gene <- wp2gene %>% tidyr::separate(ont, c("name","version","wpid","org"), "%")
wpid2gene <- wp2gene %>% dplyr::select(wpid, gene) #TERM2GENE
wpid2name <- wp2gene %>% dplyr::select(wpid, name) #TERM2NAME

ewp2 <- GSEA(geneList_NPC, 
                   pvalueCutoff = 0.05,
                   pAdjustMethod ="BH",
                   TERM2GENE = wpid2gene, 
                   TERM2NAME = wpid2name, 
                   minGSSize    = 20,
                   maxGSSize    = 700,
                   nPerm = 10000,
                   verbose=FALSE)
ewp_read <- setReadable(ewp2, "org.Mm.eg.db", keyType = "ENTREZID")

ewp2_df = as.data.frame(ewp_read)
write.table(ewp2_df,"../INTERMEDIATE_FILES/pathway_NPC48h_EPZvsDMSO.tsv", sep="\t",quote=F)


In [28]:
%%R

pdf("../FIGURES/GSEA_NPC48h_EPZvsDMSO_WIKIPATHWAY.pdf", width=7,height=5)
p = ridgeplot(ewp2, showCategory=15)
print(p)
dev.off()

/home/ferrari/.local/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Picking joint bandwidth of 0.0335

  warnings.warn(x, RRuntimeWarning)


png 
  2 
